In [1]:
import random
import math
import matplotlib.pyplot as plt
import copy
import heapq
import numpy as np

In [ ]:
N = 6  # num of vertex
M = 10  # network radius
R = 2  # neighbors radius
K = 1  # num of channels
F = 10  # number of flows
FLOW_MAX_DATA = 1000
calc_inter_face = False

CONSTANT_POWER = 1
CONSTANT_BW = 1
down_Ri = 41e5
K_B = 1.38e-23  # Boltzmann constant in Joules/Kelvin
T = 290       # Temperature in Kelvin


def set_global_params(k=None, n=None, m=None, r=None, f=None):
    global K, N, M, R, F
    K = k or 1
    N = n or 5
    M = m or 10
    R = r or 2
    F = f or 10

<h2>fundamental function

In [ ]:
def PrintRateResults(xAxis, yAxis, users, alpha, Algorithm):
    plt.figure()
    print(f"-------------------------")
    print(f"{Algorithm} Algorithm, alpha={str(alpha)} Results:")
    sum_rate = 0
    for user in users:
        sum_rate += user.rate
        print(f"user {user.Uid} rate : {round(user.rate,2)}")
    print(f"sum_rate={round(sum_rate,2)}\n")
    for i in range(len(xAxis)):
        plt.plot(xAxis[i], yAxis[i], label=f"user {i+1}")
    plt.title(f"{Algorithm} Algorithm, alpha={str(alpha)}")
    plt.xlabel("Iteration Number")
    plt.ylabel("Rate")
    plt.legend()
    plt.grid()
    plt.show(block=False)

In [ ]:
def CalcNetworkRate(network, alpha, Algorithm, N=1e5):
    network.initial_users_rates()
    algorithm_functions = {"Primal": CalcPrimalRate, "Dual": CalcDualRate}
    CalcRate = algorithm_functions.get(Algorithm)
    users = network.users

    xAxis = []
    yAxis = []
    for _ in users:  # initialize the graph
        xAxis.append([])
        yAxis.append([])

    for i in range(int(N)):
        curUser = random.choice(users)
        id = curUser.Uid-1
        x_r = curUser.rate
        curUser.rate = CalcRate(curUser, users, alpha, x_r)
        xAxis[id].append(i)
        yAxis[id].append(curUser.rate)

    PrintRateResults(xAxis, yAxis, users, alpha, Algorithm)

In [ ]:
def CalcPrimalRate(user, users, alpha, x_r, stepSize=0.0001):
    if alpha == float("inf"):
        avg_rate = sum(u.rate for u in users) / len(users)
        return max(0, min(1, avg_rate + stepSize)) if user.rate < avg_rate else max(0, user.rate - stepSize)

    payment = 0
    for link in user.links:  # calculate the payment of the user
        rateSum = 0
        for u in users:  # calculate the sum of the rates of all the users on the link
            if link in u.links:
                rateSum += u.rate
        payment += penaltyFunction(rateSum, link.total_capacity)
    return stepSize * (pow(user.rate, -1 * alpha) - payment) + x_r  # calculate the next rate of the user



In [ ]:
def penaltyFunction(rate, capacity):
    if rate < capacity:
        return rate * capacity
    else:
        try:
            return pow(rate, 3) * 2
        except OverflowError: # TODO: check why it is overflow error
            return 0



In [ ]:
def CalcDualRate(user, users, alpha, x_r, stepSize=0.0001):
    """ this function calculates the next rate of a given user for the dual algorithm """

    if alpha == float("inf"):
        # Adjusting based on the max constraint violation
        max_excess = max((sum(u.rate for u in users if link in u.links) - link.total_capacity) for link in user.links)
        return max(0, min(1, x_r - stepSize * max_excess))

    Q_l = 0
    for link in user.links:  # calculate the payment of the user
        rateSum = sum(u.rate for u in users if link in u.links) # Y_l
        L_delta = (rateSum - link.total_capacity) * stepSize
        link.LagrangianMultiplier = max(0, link.LagrangianMultiplier + L_delta)
        Q_l += link.LagrangianMultiplier
    if Q_l == 0:
        print("Ql is zero!")
    return pow(Q_l, -1/alpha) if Q_l != 0 else 0 # the inverse function of the utilization function



In [ ]:
def dijkstra_algorithm(network, start_vertex):
    # Initialize distances to all vertices in the network as infinity, except for the start vertex set to 0
    distances = {vertex: float('inf') for vertex in network.vertices}
    previous_nodes = {vertex: None for vertex in network.vertices}
    distances[start_vertex] = 0
    priority_queue = [(0, start_vertex)]  # Priority queue to manage the exploration of vertices

    # The main loop continues until there are no more vertices to explore
    while priority_queue:
        current_distance, current_vertex = heapq.heappop(priority_queue)  # Pop the vertex with the lowest distance
        if current_distance > distances[current_vertex]:  # If the popped distance is greater then known stop explore
            continue

        for neighbor, link in current_vertex.neighbors.items(): # Explore each neighbor of the current vertex
            distance = current_distance + link.weight

            if distance < distances[neighbor]:  # If the new distance is less than the previous update route
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_vertex
                heapq.heappush(priority_queue, (distance, neighbor))  # Push the updated distance for further explore

    return distances, previous_nodes



In [ ]:
def visualize_flow_rates_and_link_utilization(network, K):
    # Data for Flow Rates vs. Data Amounts
    data_amounts = [flow.data_amount for flow in network.flows]
    flow_rates = [flow.rate for flow in network.flows]
    link_ids = [link.Lid for link in network.links]

    average_utilities = []
    channel_data = {}  # Dict to hold data per channel per link

    # Collecting data
    for link in network.links:
        channel_data[link] = [0] * K  # Initialize with zero flow rate for each channel
        for user in network.users:
            if link in user.links:
                for flow in user.flows:
                    channel_data[link][flow.channel] += flow.rate

    for i, link in enumerate(network.links):
        rates = channel_data[link]
        utilities = [(rate / link.total_capacity) * 100 for rate in rates]  # Calculate utility as a percentage
        average_utility = sum(utilities)
        rounded_average_utility = int(round(average_utility))
        average_utilities.append(rounded_average_utility)

    # Calculate the overall average utility
    overall_average_utility = sum(average_utilities) / len(average_utilities)

    # Create a figure with 2 subplots
    fig, axs = plt.subplots(2, 1, figsize=(15, 7))  # 2 rows, 1 column

    # Subplot 1: Flow Rates vs. Data Amounts
    axs[0].scatter(data_amounts, flow_rates, color='red')
    axs[0].set_title('Flow Rates vs. Data Amounts')
    axs[0].set_xlabel('Data Amount')
    axs[0].set_ylabel('Flow Rate (bps)')
    axs[0].grid(True)

    # Subplot 2: Link Capacity Utilization
    colors = ['green' if percent == 100 else 'red' for percent in average_utilities]
    axs[1].bar(link_ids, average_utilities, color=colors, width=0.4)  # Adjust bar width to 0.4 for thinner bars
    axs[1].set_title('Link Capacity Utilization (%)')
    axs[1].set_xlabel('Link ID')
    axs[1].set_ylabel('Utilization (%)')
    axs[1].set_ylim(0, 100)
    axs[1].set_xticks(link_ids)
    axs[1].tick_params(axis='x', rotation=45)  # Rotate x-tick labels to prevent overlap
    axs[1].grid(True)

    # Adding a horizontal line for the overall average utility
    axs[1].axhline(y=overall_average_utility, color='blue', linestyle='--', label=f'Average Utility: {overall_average_utility:.2f}%')
    axs[1].legend(loc='upper left')

    # Display the combined plot
    plt.tight_layout()
    plt.show(block=False)


In [ ]:
def set_flows_rate_based_on_tdma(network, K):
    for link in network.links:
        flows = []
        total_data = [0] * K
        users_using_link = [user for user in network.users if link in user.links]
        for user in users_using_link:
            for flow in user.flows:
                flows.append(flow)
                total_data[flow.channel] += flow.data_amount

        for flow in flows:
            flow.rate_by_links[link] = link.channels_capacities[flow.channel] * flow.data_amount / total_data[flow.channel]

    for flow in network.flows:
        flow.set_rate_2_min_of_rate_by_links()

    visualize_flow_rates_and_link_utilization(network, K)


In [ ]:

def optimize_flows_rate_based_on_tdma(network, K):
    for link in network.links:
        flows = []
        channel_flows = {k: [] for k in range(K)}
        total_data_in_channel = [0] * K
        users_using_link = [user for user in network.users if link in user.links]
        for user in users_using_link:
            for flow in user.flows:
                flows.append(flow)
                channel_flows[flow.channel].append(flow)
                total_data_in_channel[flow.channel] += flow.data_amount

        if all(not flows for flows in channel_flows.values()):  # check if channel_flows
            continue

        while True:
            max_data, max_channel = max((value, index) for index, value in enumerate(total_data_in_channel))
            min_data, min_channel = min((value, index) for index, value in enumerate(total_data_in_channel))
            diff = max_data - min_data

            temp_total_data_in_channel = copy.copy(total_data_in_channel)
            temp_channel_flows = copy.deepcopy(channel_flows)

            min_data_flow_from_max_channel = min(temp_channel_flows[max_channel], key=lambda flow: flow.data_amount)
            temp_channel_flows[max_channel].remove(min_data_flow_from_max_channel)
            temp_total_data_in_channel[max_channel] -= min_data_flow_from_max_channel.data_amount
            temp_channel_flows[min_channel].append(min_data_flow_from_max_channel)
            temp_total_data_in_channel[min_channel] += min_data_flow_from_max_channel.data_amount

            max_data, max_channel = max((value, index) for index, value in enumerate(temp_total_data_in_channel))
            min_data, min_channel = min((value, index) for index, value in enumerate(temp_total_data_in_channel))
            new_diff = max_data - min_data
            if new_diff >= diff:
                break  # Break the loop if no improvement in fairness diff

            total_data_in_channel = temp_total_data_in_channel
            channel_flows = temp_channel_flows

        total_data_in_channel = [max(1, td) for td in total_data_in_channel]  # Ensure min data is one to guaranty rate
        total_data_in_link = sum(total_data_in_channel)
        link.channels_capacities = [link.total_capacity * (td / total_data_in_link) for td in total_data_in_channel]

        for flow in flows:
            flow.rate_by_links[link] = link.channels_capacities[flow.channel] * flow.data_amount / total_data_in_channel[flow.channel]

    for flow in network.flows:
        flow.set_rate_2_min_of_rate_by_links()

    visualize_flow_rates_and_link_utilization(network, K)

In [ ]:
def divideToCluster(numOfUsersinCluster, linkStatus, links, BW_cluser, Ptol, omega):
    arr = []
    if numOfUsersinCluster == 2:
        if linkStatus == "Up":
            cluster1 = Cluster(1, [links[0], links[6]], BW_cluser, Ptol, omega)
            cluster2 = Cluster(2, [links[1], links[7]], BW_cluser, Ptol, omega)
            cluster3 = Cluster(3, [links[2], links[8]], BW_cluser, Ptol, omega)
            cluster4 = Cluster(4, [links[3], links[9]], BW_cluser, Ptol, omega)
            cluster5 = Cluster(5, [links[4], links[10]], BW_cluser, Ptol, omega)
            cluster6 = Cluster(6, [links[5], links[11]], BW_cluser, Ptol, omega)
            arr.append(cluster1)
            arr.append(cluster2)
            arr.append(cluster3)
            arr.append(cluster4)
            arr.append(cluster5)
            arr.append(cluster6)
            return arr
        elif linkStatus == "Down":
            cluster1 = Cluster(1, [links[0], links[11]], BW_cluser, Ptol, omega)
            cluster2 = Cluster(2, [links[1], links[10]], BW_cluser, Ptol, omega)
            cluster3 = Cluster(3, [links[2], links[9]], BW_cluser, Ptol, omega)
            cluster4 = Cluster(4, [links[3], links[8]], BW_cluser, Ptol, omega)
            cluster5 = Cluster(5, [links[4], links[7]], BW_cluser, Ptol, omega)
            cluster6 = Cluster(6, [links[5], links[6]], BW_cluser, Ptol, omega)
            arr.append(cluster1)
            arr.append(cluster2)
            arr.append(cluster3)
            arr.append(cluster4)
            arr.append(cluster5)
            arr.append(cluster6)
            return arr  ;

    elif numOfUsersinCluster == 3:
        if linkStatus == "Up":
            cluster1 = Cluster(1, [links[0], links[4], links[8]], BW_cluser, Ptol, omega)
            cluster2 = Cluster(2, [links[1], links[5], links[9]], BW_cluser, Ptol, omega)
            cluster3 = Cluster(3, [links[2], links[6], links[10]], BW_cluser, Ptol, omega)
            cluster4 = Cluster(4, [links[3], links[7], links[11]], BW_cluser, Ptol, omega)
            arr.append(cluster1)
            arr.append(cluster2)
            arr.append(cluster3)
            arr.append(cluster4)
            return arr
        elif linkStatus == "Down":
            cluster1 = Cluster(1, [links[0], links[4], links[11]], BW_cluser, Ptol, omega)
            cluster2 = Cluster(2, [links[1], links[5], links[10]], BW_cluser, Ptol, omega)
            cluster3 = Cluster(3, [links[2], links[6], links[9]], BW_cluser, Ptol, omega)
            cluster4 = Cluster(4, [links[2], links[7], links[8]], BW_cluser, Ptol, omega)
            arr.append(cluster1)
            arr.append(cluster2)
            arr.append(cluster3)
            arr.append(cluster4)
            return arr

    elif numOfUsersinCluster == 4:
        if linkStatus == "Up":
            cluster1 = Cluster(1, [links[0], links[3], links[6], links[9]], BW_cluser, Ptol, omega)
            cluster2 = Cluster(2, [links[1], links[4], links[7], links[10]], BW_cluser, Ptol, omega)
            cluster3 = Cluster(3, [links[2], links[5], links[8], links[11]], BW_cluser, Ptol, omega)
            arr.append(cluster1)
            arr.append(cluster2)
            arr.append(cluster3)
            return arr
        elif linkStatus == "Down":
            cluster1 = Cluster(1, [links[0], links[3], links[8], links[11]], BW_cluser, Ptol, omega)
            cluster2 = Cluster(2, [links[1], links[4], links[7], links[10]], BW_cluser, Ptol, omega)
            cluster3 = Cluster(3, [links[2], links[5], links[6], links[9]], BW_cluser, Ptol, omega)
            arr.append(cluster1)
            arr.append(cluster2)
            arr.append(cluster3)
            return arr

    raise ValueError(f"undefined_scenario:\n numOfUsersinCluster:{numOfUsersinCluster}, linkStatus:{linkStatus}")


In [ ]:

def compare_OMA_NOMA_rates(network):
    # Parameters
    cluster_sizes = [2, 3, 4]  # User cluster sizes
    link_statuses = ['Up', 'Down']

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6), sharey=True)

    for index, link_status in enumerate(link_statuses):
        OmaRate = network.Calculate_OMA_rates(link_status)
        Oma_rates = [OmaRate] * len(cluster_sizes)
        NomaRates = []
        for num_of_users in cluster_sizes:
            NomaRates.append(network.Calculate_NOMA_rates(num_of_users, link_status))

        # Plotting
        axes[index].plot(cluster_sizes, Oma_rates, 'r--', label='OMA Rate',
                         linewidth=2)  # OMA rates as a red dashed line
        axes[index].scatter(cluster_sizes, NomaRates, color='blue', label='NOMA Rates', s=100,
                            zorder=5)  # NOMA rates as points
        axes[index].set_title(f'{link_status} Link Rates')
        axes[index].set_xlabel('Number of Users in Cluster')
        axes[index].set_ylabel('Rate (Mbps)')
        axes[index].set_xticks(cluster_sizes)  # Set X-ticks to only include the cluster sizes
        axes[index].legend()
        axes[index].grid(True)

    plt.tight_layout()
    plt.show(block=False)

<h2>calculate_path_loss</h2>
aim: Calculate the path loss using the log-distance path loss model.

distance: The distance between the transmitter and receiver in meters.
path_loss_exponent: The path loss exponent, depending on the environment.
PL0: The path loss at a reference distance (typically 1 meter) in dB.
return: The path loss in dB.

$$
\text{PL} = PL_0 + 10 \times \gamma \times \log_{10}(\text{distance})
$$

where:

- PL is the path loss in dB.
- PL0 is the path loss at the reference distance (typically 1 meter), default value is -30 dB.
- gamma is the path loss exponent, default value is 3.5.
- distance- is the distance between the transmitter and receiver in meters.

In [ ]:

def calculate_path_loss(distance, path_loss_exponent=3.5, PL0=-30):
    if distance <= 0:
        raise ValueError("Distance must be greater than 0")
    path_loss = PL0 + 10 * path_loss_exponent * np.log10(distance)
    return path_loss



<h2>rayleigh fading</h2>
function simulates small-scale fading using the Rayleigh distribution. Small-scale fading refers to the rapid fluctuations of the amplitude, phase, or multipath delays of a radio signal over a short period or travel distance. The Rayleigh distribution is commonly used to model such fading, particularly in environments without a line of sight (NLOS) between the transmitter and receiver, where there are many obstacles causing reflections.

In [ ]:
def rayleigh_fading():
    """
    Simulate small scale fading using Rayleigh distribution.
    :return: A random fading coefficient sampled from a Rayleigh distribution.
    """
    return np.random.rayleigh()

<h2>Class Vertex: </h2>
<h3>Purpose:</h3>
The Vertex class represents a node in a network graph. Each vertex has a unique identifier and a random location within a specified area. The class also manages the vertex's neighbors, power, bandwidth, and shortest paths to other vertices.

<h3>Attributes:</h3>
Vid: A unique identifier for the vertex.
location: A tuple representing the random location of the vertex within a circle of radius 
𝑀.
neighbors: A dictionary storing neighboring vertices and the links connecting them.
ShortestPath: A dictionary storing the shortest paths to other vertices.
power: An attribute representing the power of the vertex- first initialization 0.
bw: An attribute representing the bandwidth of the vertex.

<h3>Functions:</h3>
generate_location(self): Generates a random location for the vertex within a circle of radius 
𝑀.
distance_to(self, other): Calculates the Euclidean distance between the current vertex and another vertex.
calc_neighbors(self, others, r): Calculates and returns the neighbors of the current vertex within a specified distance 𝑟.
add_neighbors(self, neighbor_vertex, connected_link): Adds a neighboring vertex and the link connecting them to the neighbors dictionary.

In [ ]:
class Vertex:
    """ this class represent a (represent a node in a graph) vertex in the network """

    def __init__(self, Vid):
        self.Vid = Vid #Unique identifier for the vertex.
        self.location = self.generate_location() #method to assign a random location to the vertex.
        self.neighbors = {} #A dictionary to store neighboring vertices and the links connecting them.
        self.ShortestPath = {}  # Dictionary to store shortest paths to other vertices
        self.power = 0
        self.bw = 0 #bandwidth

    def __str__(self):
        return str(self.Vid)

    def __lt__(self, other):
        return self.Vid < other.Vid

    def __eq__(self, other):
        return self.Vid == other.Vid

    def __hash__(self):
        return hash(self.Vid)

    def __deepcopy__(self, memo):
        cls = self.__class__
        result = cls.__new__(cls)
        memo[id(self)] = result
        result.Vid = self.Vid
        result.location = copy.deepcopy(self.location, memo)
        result.power = self.power
        result.bw = self.bw
        result.neighbors = {copy.deepcopy(key, memo): copy.deepcopy(value, memo) for key, value in
                            self.neighbors.items()}
        result.ShortestPath = {copy.deepcopy(key, memo): copy.deepcopy(value, memo) for key, value in
                               self.ShortestPath.items()}
        return result

    def generate_location(self):
        """
        Generates a random location within a circle of radius M centered at the origin.
        """
        theta = random.uniform(0, 2 * math.pi)  # Angle for circular distribution
        r = M * math.sqrt(random.uniform(0, 1))  # Distance from the center, sqrt for uniform distribution within the circle
        x = round(r * math.cos(theta), 2)
        y = round(r * math.sin(theta), 2)
        return x, y

    def distance_to(self, other):
        """
        Calculates the Euclidean distance between two vertices.
        """
        x1, y1 = self.location
        x2, y2 = other.location
        distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

        return distance

    def calc_neighbors(self, others, r):
        neighbors = []
        for other in others:
            distance = self.distance_to(other)
            if 0 < distance <= r:
                neighbors.append((other, distance))

        return neighbors

    def add_neighbors(self, neighbor_vertex, connected_link):
        self.neighbors[neighbor_vertex] = connected_link



<h2>Class Link:</h2>
<h3>Purpose:</h3>
The Link class represents a link in a network, connecting two vertex (nodes). It calculates various properties of the link, such as gain, power, interference, and capacity, based on the vertices it connects and other parameters.

<h3>Attributes:</h3>
Lid: A unique identifier for the link.
connected_vertices: A tuple containing the two vertices connected by the link (start, end).
distance: The distance between the connected vertices.
LagrangianMultiplier: A parameter for optimization purposes, defaulting to 0.5.
gain: The calculated gain of the link, determined by path loss and fading.
power: The calculated power of the link.
interference_gain: The calculated interference gain of the link.
interference_power: The interference power, initialized to 0.
total_capacity: The total capacity of the link.
channels_capacities: A list of channel capacities, initially set to equal parts of the total capacity divided by the number of channels K.
weight: The calculated weight of the link, based on its capacity.

<h3>Functions:</h3>
calculate_gain(self): Calculates the gain of the link based on the distance, path loss, and fading coefficient.

$$
\text{channel gain} = 10^{\left(\frac{\text{path loss}}{10}\right)}*{\text{rayleigh fading}}
$$

calculate_interference_gain(self): Calculates the interference gain of the link based on the distance, path loss (PL[db]), and fading coefficient, adjusted with a constant factor.

calculate_link_power(self): Calculates the power of the link based on the minimum power of the connected vertices and the gain.

$$
P_{\text{link}} = \min(P_{v1}, P_{v2}) \times G^2
$$

where:

- P_link is the link power.
- Pv1 is the power of the first vertex.
- Pv2 is the power of the second vertex.
- G is the gain of the link.

calculate_capacity(self): Calculates the capacity of the link based on the bandwidth of the connected vertices, noise power, and signal-to-interference-plus-noise ratio (SINR).
The function `calculate_capacity` calculates the capacity of a link using the following formula:

$$
C = B \times \log_2 \left( 1 + \text{SINR} \right)
$$

where:

- C is the capacity of the link.
- B is the bandwidth, defined as the minimum bandwidth of the two connected vertices:
  $$
  B = \min(\text{bw}_1, \text{bw}_2)
  $$
- \( \text{SINR} \) is the Signal-to-Interference-plus-Noise Ratio, calculated as:
  $$
  \text{SINR} = \frac{P_{\text{link}}}{N + I}
  $$
  where:
  - \( P_{\text{link}} \) is the power of the link.
  - \( N \) is the noise power, calculated as:
    $$
    N = K_B \times T \times B
    $$
    - \( K_B \) is the Boltzmann constant.
    - \( T \) is the temperature in Kelvin.
    - \( B \) is the bandwidth.
  - \( I \) is the interference power.

calculate_link_weight(self): Calculates the weight of the link as the inverse of its total capacity.

update_total_capacity(self): Updates the total capacity of the link by summing the capacities of its channels.

In [ ]:
class Link:
    """ this class represent a link in the network """

    def __init__(self, Lid, vertex1: Vertex = None, vertex2: Vertex = None, LagrangianMultiplier=0.5, distance=None):
        self.Lid = Lid
        self.connected_vertices = (vertex1, vertex2)
        self.distance = distance
        self.LagrangianMultiplier = LagrangianMultiplier
        self.gain = self.calculate_gain()
        self.power = self.calculate_link_power()
        self.interference_gain = self.calculate_interference_gain()
        self.interference_power = 0
        self.total_capacity = self.calculate_capacity()
        self.channels_capacities = [self.total_capacity / K for _ in range(K)]
        self.weight = self.calculate_link_weight()


    def __str__(self):
        return str(self.Lid)

    def calculate_gain(self):
        distance = self.distance
        path_loss_db = calculate_path_loss(distance)
        fading_coefficient = rayleigh_fading()
        attenuation_factor = 10 ** (path_loss_db / 10)
        gain = attenuation_factor * fading_coefficient
        return gain

    def calculate_interference_gain(self):
        distance = self.distance
        path_loss_db = calculate_path_loss(distance)
        fading_coefficient = rayleigh_fading()
        attenuation_factor = 10 ** (path_loss_db / 10)
        gain = attenuation_factor * fading_coefficient*1e-23
        return gain

    def calculate_link_power(self):
        min_vertex_power = min(self.connected_vertices[0].power, self.connected_vertices[1].power)
        power = min_vertex_power * (self.gain ** 2)
        return power

    def calculate_capacity(self):
        bw = min(self.connected_vertices[0].bw, self.connected_vertices[1].bw)
        noise_power = K_B * T * bw
        SINR = self.power / (noise_power + self.interference_power)  # Simplified SINR calculation
        capacity = bw * math.log2(1 + SINR)
        if capacity <= 0:
            raise ValueError(f"Invalid capacity {capacity} for link {self.Lid}. Capacity must be greater than zero.")
        return capacity

    def calculate_link_weight(self):
        if self.total_capacity == 0:
            return float('inf')
        return 1 / self.total_capacity

    def update_total_capacity(self):
        self.total_capacity = sum(self.channels_capacities)

<h2>Class User</h2>
<h3>Purpose:</h3>
Represents a user in the network who establishes a connection between vertices and manages data flows.

<h3>Attributes:</h3>
Uid: A unique identifier for the user.
links: A list of links that the user uses to connect from the start vertex to the end vertex.
defaultLinks: A list to store default links (initialized as an empty list).
rate: The data transfer rate for the user, defaulting to 0.001.
startVertex: The starting vertex (node) for the user's connection.
endVertex: The ending vertex (node) for the user's connection.
flows: A list to store data flows associated with the user.

<h3>Functions:</h3>
add_flow(self, flow): Adds a data flow to the user's list of flows.

In [ ]:
class User:
    def __init__(self, Uid, links=[], startVertex=None, endVertex=None, rate=0.001):
        self.Uid = Uid
        self.links = links
        self.defualtLinks = []
        self.rate = rate
        self.startVertex = startVertex
        self.endVertex = endVertex
        self.flows = []

    def __str__(self):
        string = f"\nUser({self.Uid}) connect {self.startVertex.Vid} to {self.endVertex.Vid} through: "
        for link in self.links:
            string += f"{link} "
        string += f", and sends {len(self.flows)} flow with {sum(flow.data_amount for flow in self.flows)} data"
        return string

    def add_flow(self,flow):
        self.flows.append(flow)